In [4]:
import os
import json
import torch

args_perm_id = 0
args_task_num = 5
args_class_num = 10
args_shot_num = 5
def collect_from_json(dataset, root, split):
    if split == "train":
        pth = os.path.join(
            root,
            dataset,
            "perm" + str(args_perm_id),
            f"{dataset}_{args_task_num}task_{args_class_num // args_task_num}way_{args_shot_num}shot.{split}.jsonl",
        )
    elif split in ["dev", "test"]:
        pth = os.path.join(root, dataset, f"{dataset}.{split}.jsonl")
    elif split == "stream":
        pth = os.path.join(
            root,
            dataset,
            f"stream_label_{args_task_num}task_{args_class_num // args_task_num}way.json",
        )
    else:
        raise ValueError(f'Split "{split}" value wrong!')
    if not os.path.exists(pth):
        raise FileNotFoundError(f"Path {pth} do not exist!")
    else:
        with open(pth) as f:
            if pth.endswith(".jsonl"):
                data = [json.loads(line) for line in f]
                if split == "train":
                    data = [list(i.values()) for i in data]
            else:
                data = json.load(f)
    # if split == "train":
    #     data = extract_single_dict(data)

    return data

In [8]:
dataset = 'ACE'
root = './data_incremental'
split = 'train'

In [24]:
data = [instance for t in collect_from_json(dataset, root, split)[1] for instance in t]

In [64]:
import torch


def get_one_hot_true_label_and_true_trigger(data_instance, num_label):
    true_label = []
    trigger_word = []
    seq_len = len(data_instance["piece_ids"]) + 1 # because start_index of piece_ids is 1 instead of 0
    for i in range(len(data_instance["label"])):
        if data_instance["label"][i] != 0:
            true_label.append(data_instance["label"][i])
            trigger_word.append(data_instance["span"][i])

    set_label_in_one_sentence = set(true_label)
    true_one_hot_trigger_vector = torch.zeros(num_label)
    for i in set_label_in_one_sentence:
        true_one_hot_trigger_vector += torch.eye(num_label)[i]

    true_one_hot_label_vector = torch.zeros(seq_len)
    trigger = []
    for i in trigger_word:
        trigger.extend(i)

    set_trig = set(trigger)
    for i in set_trig:
        true_one_hot_label_vector += torch.eye(seq_len)[i]
    return true_one_hot_trigger_vector, true_one_hot_label_vector

In [65]:
a, b = get_one_hot_true_label_and_true_trigger(data[0],10)

In [67]:
a

tensor([0., 1., 0., 0., 0., 0., 1., 0., 0., 0.])

In [68]:
b

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.])

In [52]:
print(len(data[0]['piece_ids']))
print(len(data[0]['label']))
print(len(data[0]['span']))

50
47
47


In [72]:
for instance in data:
    true_one_hot_trigger_vector, true_one_hot_label_vector = get_one_hot_true_label_and_true_trigger(instance, 10)
    instance['true_one_hot_trigger_vector'] = true_one_hot_trigger_vector.tolist()
    instance['true_one_hot_label_vector'] = true_one_hot_label_vector.tolist()

In [74]:
data[0]

{'piece_ids': [101,
  3960,
  1010,
  1045,
  2228,
  2008,
  1996,
  3114,
  7955,
  1999,
  1996,
  2148,
  1011,
  1011,
  2017,
  2113,
  1010,
  2034,
  1997,
  2035,
  1010,
  2057,
  2020,
  1011,
  1011,
  2043,
  5951,
  8573,
  2001,
  2700,
  2343,
  1010,
  2057,
  2018,
  2042,
  2542,
  2054,
  2057,
  2245,
  2001,
  2145,
  1037,
  11438,
  3842,
  2044,
  1996,
  2942,
  2162,
  1012,
  102],
 'label': [6,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'span': [[29, 29],
  [46, 47],
  [1, 1],
  [2, 2],
  [3, 3],
  [4, 4],
  [5, 5],
  [6, 6],
  [7, 7],
  [8, 8],
  [9, 9],
  [10, 10],
  [11, 11],
  [12, 12],
  [13, 13],
  [14, 14],
  [15, 15],
  [16, 16],
  [17, 17],
  [18, 18],
  [19, 19],
  [20, 20],
  [21, 21],
  [22, 22],
  [23, 23],
  [24, 24],
  [25, 25],
  [26, 26],
  [27, 27],
  [2

In [75]:
torch.cuda.is_available()

False

In [151]:
import torch


def get_one_hot_true_label_and_true_trigger(data_instance, num_label):
    true_label = []
    true_trigger = []
    seq_len = len(data_instance["piece_ids"]) # because start_index of piece_ids is 1 instead of 0
    
    for i in range(len(data_instance["label"])):
        if data_instance["label"][i] != 0:
            true_label.append(data_instance["label"][i])
            true_trigger.append(data_instance["span"][i])


    true_one_hot_label_vector = torch.zeros(num_label)
    true_one_hot_trigger_vector = torch.zeros(seq_len)

    set_label_in_one_sentence = set([label.item() for label in true_label])
    for i in set_label_in_one_sentence:
        true_one_hot_label_vector += torch.eye(num_label)[i]


    set_trig = [trigger.tolist() for trigger in true_trigger]
    trigger = []
    for i in set_trig:
        trigger.extend(i)
    trigger = set(trigger)

    for i in set_trig:
        true_one_hot_trigger_vector += torch.eye(seq_len)[i]
    
    return set_trig, set_label_in_one_sentence
    # return true_one_hot_trigger_vector, true_one_hot_label_vector

In [152]:
import torch
def true_label_and_trigger(train_x,train_y,train_masks, train_span, class_num):
    num_instance = len(train_x)
    true_one_hot_label_vectors = []
    true_one_hot_trigger_vectors = []
    for i in range(num_instance):
        data_instace={
            'piece_ids': train_x[i],
            'label': train_y[i],
            'span': train_span[i],
            'mask': train_masks[i]
        }

        true_one_hot_trigger_vector, true_one_hot_label_vector = get_one_hot_true_label_and_true_trigger(data_instance=data_instace,num_label=class_num)
        true_one_hot_trigger_vectors.append(true_one_hot_trigger_vector)
        true_one_hot_label_vectors.append(true_one_hot_label_vector)
    return true_one_hot_trigger_vectors, true_one_hot_label_vectors

        

In [149]:
train_x = torch.tensor([[  101,  6398,  1024,  6175,  2003,  2025,  1996,  2069,  2510,  2564,
          2040,  2363,  1037,  6302,  1998,  3661,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101, 21524,  1998,  2037,  2079, 24968,  5611,  6956, 19974,  2224,
          1996,  2773,  1036,  1036,  6139,  1005,  1005,  2043,  9694,  2008,
          3956,  2681,  1996,  2225,  2924,  1998, 14474,  1998,  4487, 11512,
          9286,  3644,  7617,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1045,  2079,  1050,  1005,  1056,  2228,  2008,  1005,  1055,
          3243,  8321,  2004,  2000,  2054,  2002,  2626,  2033,  2055,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  2358, 23111,  6582,  1010,  2040,  2001,  2809,  2706,  6875,
          1010,  2018,  3041,  2042,  3331,  2007,  2014,  2388,  2006,  1996,
          3042,  1010,  1998,  5112,  2039,  3038,  1037,  2450,  2016,  2018,
         11834,  3064,  2007,  3784,  2018,  2074,  3369,  2012,  2014,  2341,
          1010,  4614,  2056,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])

train_y = [torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), torch.tensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]), torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), torch.tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
train_masks = torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])
train_span = [torch.tensor([[15, 15],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14],
        [16, 16]]), torch.tensor([[21, 21],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14],
        [15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [20, 20],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28],
        [29, 29],
        [30, 30],
        [31, 31],
        [32, 32],
        [33, 33]]), torch.tensor([[16, 16],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14],
        [15, 15],
        [17, 17],
        [18, 18],
        [19, 19]]), torch.tensor([[20, 20],
        [30, 31],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14],
        [15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [21, 21],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28],
        [29, 29],
        [32, 32],
        [33, 33],
        [34, 34],
        [35, 35],
        [36, 36],
        [37, 37],
        [38, 38],
        [39, 39],
        [40, 40],
        [41, 41],
        [42, 42],
        [43, 43]])]

In [92]:
true_trigger, true_label = true_label_and_trigger(train_x=train_x,train_y=train_y,train_masks=train_masks,train_span=train_span,class_num=10)

In [93]:
true_trigger

[tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 2., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [150]:
data_instance = {
    "piece_ids": train_x[3],
    "label": train_y[3],
    "span": train_span[3],
    "mask": train_masks[3],
}
a, b = get_one_hot_true_label_and_true_trigger(data_instance=data_instance,num_label=10)

RuntimeError: output with shape [122] doesn't match the broadcast shape [2, 122]

In [154]:
data_instance

{'piece_ids': tensor([  101,  2358, 23111,  6582,  1010,  2040,  2001,  2809,  2706,  6875,
          1010,  2018,  3041,  2042,  3331,  2007,  2014,  2388,  2006,  1996,
          3042,  1010,  1998,  5112,  2039,  3038,  1037,  2450,  2016,  2018,
         11834,  3064,  2007,  3784,  2018,  2074,  3369,  2012,  2014,  2341,
          1010,  4614,  2056,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]),
 'label': tens

In [166]:

true_label = []
true_trigger = []
num_label = 10
seq_len = len(data_instance["piece_ids"]) # because start_index of piece_ids is 1 instead of 0

for i in range(len(data_instance["label"])):
    if data_instance["label"][i] != 0:
        true_label.append(data_instance["label"][i])
        true_trigger.append(data_instance["span"][i])


true_one_hot_label_vector = torch.zeros(num_label)
true_one_hot_trigger_vector = torch.zeros(seq_len)

# # set_label_in_one_sentence = set([label.item() for label in true_label])
# # for i in set_label_in_one_sentence:
# #     true_one_hot_label_vector += torch.eye(num_label)[i]


# set_trig = [trigger.tolist() for trigger in true_trigger]
# trigger = []
# for i in set_trig:
#     trigger.extend(i)
# trigger = set(trigger)

# for i in set_trig:
#     true_one_hot_trigger_vector += torch.eye(seq_len)[i]




tensor([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [158]:
torch.eye(seq_len)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [146]:
set(l)

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43}

In [128]:
t.tolist()

[1, 2]

In [109]:
true_label = []
trigger_word = []
seq_len = len(data_instance["piece_ids"]) # because start_index of piece_ids is 1 instead of 0
for i in range(len(data_instance["label"])):
    if data_instance["label"][i] != 0:
        true_label.append(data_instance["label"][i])
        trigger_word.append(data_instance["span"][i])    

In [116]:
set_label_in_one_sentence = set([label.item() for label in true_label])
set_label_in_one_sentence

{1}

In [119]:
set_label_in_one_sentence = set([label.item() for label in true_label])
true_one_hot_trigger_vector = torch.zeros(10)
for i in set_label_in_one_sentence:
    true_one_hot_trigger_vector += torch.eye(10)[i]

# true_one_hot_label_vector = torch.zeros(seq_len)
# trigger = []
# for i in trigger_word:
#     trigger.extend(i)

# set_trig = set(trigger)
# for i in set_trig:
#     true_one_hot_label_vector += torch.eye(seq_len)[i]

In [120]:
true_one_hot_trigger_vector

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])